In [204]:
import pandas as pd
import json
import requests

url = "https://raw.githubusercontent.com/Papagoat/brain-assessment/main/restaurant_data.json"  # Replace with your actual JSON URL
response = requests.get(url)
with open('restaurant_data.json', 'w') as json_file:  
    json.dump(response.json(), json_file, indent=4) 


In [92]:
def preprocess_country_codes():
    country_code_df = pd.read_excel('Country-Code.xlsx', sheet_name='Sheet1')
    return country_code_df


In [165]:
def append_restaurant_details(restaurant, restaurant_ids, restaurant_names, restaurant_countrycodes, restaurant_cities, restaurant_votes, restaurant_user_ratings, restaurant_cuisines):
    restaurant_ids.append(restaurant.get("R").get("res_id"))
    restaurant_names.append(restaurant.get("name"))
    restaurant_countrycodes.append(restaurant.get("location").get("country_id"))
    restaurant_cities.append(restaurant.get("location").get("city"))
    restaurant_votes.append(restaurant.get("user_rating").get("votes"))
    restaurant_user_ratings.append(restaurant.get("user_rating").get("aggregate_rating"))
    restaurant_cuisines.append(restaurant.get("cuisines"))

In [161]:
def preprocess_restaurant_data():
    with open("restaurant_data.json") as res:
        data = json.load(res)

    restaurant_ids = []
    restaurant_names = []
    restaurant_countrycodes = []
    restaurant_cities = []
    restaurant_votes = []
    restaurant_user_ratings = []   
    restaurant_cuisines = []
    restaurant_eventids = []
    restaurant_photourls = []
    restaurant_eventtitles = []
    restaurant_eventstartdates = []
    restaurant_eventenddates = []

    for result in data:
        for r in result["restaurants"]:
            restaurant = r["restaurant"]

            append_restaurant_details(restaurant, restaurant_ids, restaurant_names, restaurant_countrycodes, restaurant_cities, restaurant_votes, restaurant_user_ratings, restaurant_cuisines)

            event_results = restaurant.get("zomato_events", "NA")

            if event_results == "NA" :
                [lst.append("NA") for lst in [restaurant_eventids, restaurant_photourls, restaurant_eventstartdates, restaurant_eventenddates, restaurant_eventtitles]]
            
            else:
                for i, e in enumerate(event_results):
                    if i != 0:
                        append_restaurant_details(restaurant, restaurant_ids, restaurant_names, restaurant_countrycodes, restaurant_cities, restaurant_votes, restaurant_user_ratings, restaurant_cuisines)
                    event = e.get("event")
                    restaurant_eventids.append(event.get("event_id", "NA"))
                    if "photos" in event and len(event.get("photos")) != 0:
                            restaurant_photourls.append(event.get("photos")[0].get("photo").get("url", "NA"))
                    else:
                        restaurant_photourls.append("NA")
                    restaurant_eventstartdates.append(event.get("start_date", "NA"))
                    restaurant_eventenddates.append(event.get("end_date", "NA"))
                    restaurant_eventtitles.append(event.get("title", "NA"))

    data = {"Restaurant Id" : restaurant_ids, 
            "Restaurant Name" : restaurant_names, 
            "Country Code" : restaurant_countrycodes,
            "City" : restaurant_cities,
            "User Rating Votes" : restaurant_votes,
            "User Aggregate Rating" : restaurant_user_ratings,
            "Cuisines" : restaurant_cuisines,
            "Event Id" : restaurant_eventids,
            "Photo URL" : restaurant_photourls,
            "Event Title" : restaurant_eventtitles,
            "Event Start Date" : restaurant_eventstartdates,
            "Event End Date" : restaurant_eventenddates
    }

    df = pd.DataFrame(data)
    return df





In [162]:
def output_restaurant_data(restaurant_data_df, country_code_df):
    merged_df = restaurant_data_df.merge(country_code_df, on = "Country Code", how = "left")
    merged_df = merged_df.drop(columns=["Country Code"])
    merged_df = merged_df[["Restaurant Id", "Restaurant Name", "Country", "City", "User Rating Votes", "User Aggregate Rating", "Cuisines"]]
    merged_df.to_csv("restaurant_data.csv", index = False)

In [206]:
def output_restaurant_events(restaurant_data_df):
    filtered_df = restaurant_data_df[restaurant_data_df["Event Start Date"] != "NA"]
    filtered_df["converted_eventstartdt"] = pd.to_datetime(filtered_df["Event Start Date"])
    filtered_df1 = filtered_df.loc[lambda x : (x["converted_eventstartdt"].dt.year == 2019) & (x["converted_eventstartdt"].dt.month == 4)]
    filtered_df2 = filtered_df1[["Event Id", "Restaurant Id", "Restaurant Name", "Photo URL", "Event Title", "Event Start Date", "Event End Date"]]
    filtered_df2.to_csv("restaurant_events.csv", index = False)

In [207]:
if __name__ == "__main__":
    restaurant_data_df = preprocess_restaurant_data()
    country_code_df = preprocess_country_codes()
    output_restaurant_data(restaurant_data_df, country_code_df)
    output_restaurant_events(restaurant_data_df)

/var/folders/1f/s28t3bkj1978dtt71g0gbvqw0000gn/T/ipykernel_24925/3878880548.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  filtered_df["converted_eventstartdt"] = pd.to_datetime(filtered_df["Event Start Date"])
